In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# pip install pgzip==0.3.0

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13284795341190200264
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3680501760
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15677916054351649379
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [4]:
import numpy as np

import sys
import time
sys.path.append("./Models")
import os
os.system('')

import subprocess
import torch
from torch.utils.data import Dataset

import pickle
import pgzip
import copy

import datetime

import math
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt

In [5]:
#Default settings for experiment
arg_model = "tsrnn" #Options: 'trfbb', 'tsrnn', 'trfbf'
arg_dset = "dyt" #Datasets -- Spain: 'ree', AEP, DAYTON: 'dyt' London: 'lsm'

attr_dset_smpl_rt = 24 if arg_dset == "aep" else (48 if arg_dset == "lsm" else 24) #Samples per day. Spain, AEP: 24, London: 48
param_dset_lookback_weeks = 5
param_dset_forecast = 48 if arg_dset == "lsm" else 24
# param_dset_lookback_weeks = 9
# param_dset_forecast = 168 if arg_dset == "lsm" else 84 # 3.5days = 168
param_dset_train_stride = 48 #Choose a coprime value to the forecast so all reading frames are eventually considered
param_dset_test_stride = 'same' #tsrnn paper uses 1 week
param_dset_lookback = param_dset_lookback_weeks*7*attr_dset_smpl_rt - param_dset_forecast

param_trf_weather = False



In [6]:
import pywt
from tqdm.auto import tqdm

# Get the maximum decomposition level
def print_maximal_decom_level(data):
    max_level = pywt.swt_max_level(data)
    print("Maximum decomposition level:", max_level)

    return max_level

# SWT functions
def data_preparation(dataset, window, lev):
    da = []
    max_level = print_maximal_decom_level(window)
    for i in tqdm(range(len(dataset)), total=len(dataset), desc="swt"):
        coeffs = pywt.swt(dataset[i], wavelet='db2', level=lev)
        da.append(coeffs);
    return da

def data_reconstruction(dataset):
    da = []
    for i in tqdm(range(len(dataset)), total= len(dataset), desc="iswt"):
#         recon = pywt.iswt(dataset[i,:,:,:].tolist(), 'db2')
        recon = pywt.iswt(dataset[i], 'db2')
#         print(np.array(recon).shape)
        da.append(recon)
#         da.append(recon[0][window-1])
    return da


# Called because iswt cannot accept tolist() dataset
def data_organization(coeffs):
    '''
    Reshape data back to (n,3,2,window_length), where there are 3 tuples of 2 values consisting of
    coeffs array_like Coefficients list of tuples:
    [(cAn, cDn), ..., (cA2, cD2), (cA1, cD1)]
    '''
    reshape_list = []
    for i in range(len(coeffs)):
        reshape_list.append([])
        for j in range(len(coeffs[0])):
            reshape_list[i].append(tuple(coeffs[i][j]))

    return reshape_list

In [7]:
# -*- coding: utf-8 -*-

import numpy as np

import os
import subprocess
import torch
from torch.utils.data import Dataset

import pickle
import pgzip
import copy

import datetime

class DAYTON(Dataset):
    """
    PJM-DAYTON dataset. Instantaneous power reported in MW.
    Ranges from around ??? GW (???? MW) to ??? GW (???? MW).
    """
    def __init__(self,path = '.',start_idx = 0, end_idx = 9999999,
                 seq_len = 816, pred_horz = 24, stride=-1, timestamp = True):
        assert(end_idx - start_idx > seq_len+pred_horz)
        if stride <= 0:
            stride = seq_len+pred_horz
        #if 'aep_tensor.pkl' not in os.listdir(path) or 'aep_timestamps.pkl' not in os.listdir(path):
        if 'dayton_dict.pkl.pgz' not in os.listdir(path):
            raise FileNotFoundError(os.listdir(path))
            #subprocess.check_call('python ./dayton_.py . aep_dict.pkl.pgz')

        # with open(os.path.join(path,'aep_tensor.pkl'),'rb') as f:
        #     series = pickle.load(f)
        # with open(os.path.join(path,'aep_timestamps.pkl'),'rb') as f:
        #     timestamps = pickle.load(f)

        with pgzip.open(os.path.join(path,'dayton_dict.pkl.pgz')) as f:
            sd = pickle.load(f)

        series = sd["tensor"] #121296 total elements
        starttime = sd['start_time']

        self.total_datapoints = len(series)

        end_idx = min(end_idx,len(series))

        wset = []
        ser_start = []
        pred_start = []
        i = start_idx
        while i + seq_len + pred_horz < end_idx:
            wset.append(series[i:i+seq_len+pred_horz])
            # ser_start.append(timestamps[i])
            # pred_start.append(timestamps[i+seq_len+1])
            ser_start.append(starttime + i*datetime.timedelta(hours=1))
            pred_start.append(starttime + (i + seq_len)*datetime.timedelta(hours=1))
            i += stride

        wset = torch.stack(wset).unsqueeze(-1)
        self.series = wset

        for i in range(len(ser_start)):
            tmptime = ser_start[i]
            ser_start[i] = [tmptime.year,
                            tmptime.month,
                            tmptime.day,
                            tmptime.hour,
                            tmptime.minute,
                            tmptime.second]

        for i in range(len(pred_start)):
            tmptime = pred_start[i]
            pred_start[i] = [tmptime.year,
                            tmptime.month,
                            tmptime.day,
                            tmptime.hour,
                            tmptime.minute,
                            tmptime.second]

        #Series normalization
        #Convert nans so that they do not count toward the min/max
        # smin = self.series.nan_to_num(nan=torch.finfo(self.series.dtype).max).amin(dim=-2,keepdim=True)
        # smax = self.series.nan_to_num(nan=torch.finfo(self.series.dtype).min).amax(dim=-2,keepdim=True)

        self._min = self.series[~self.series.isnan()].min()
        self._max = self.series[~self.series.isnan()].max()

        #self.series = (self.series - smin.broadcast_to(self.series.shape))/(smax-smin).broadcast_to(self.series.shape)
        self.series_starttimes = torch.tensor(ser_start,dtype=torch.long)
        self.pred_start = torch.tensor(pred_start,dtype=torch.long)

        self.length = len(self.series)
        self.return_timestamps = timestamp

        self.seq_len = seq_len
        self.pred_horz = pred_horz

        #Dataset in use will contain nans, to be handled by the training code

    def __len__(self):
        return self.length

    def min(self):
        """Returns the minimum load power in GW"""
        return self._min
        #return 18.041

    def max(self):
        """Returns the maximum load power in GW"""
        return self._max
        #return 41.015

    def __getitem__(self,idx):
        if self.return_timestamps:
            return self.series[idx][:self.seq_len], self.series[idx][self.seq_len:], self.series_starttimes[idx], self.pred_start[idx]
        return self.series[idx][:self.seq_len], self.series[idx][self.seq_len:]

In [8]:
# path = '/content/gdrive/MyDrive/FYP/Datasets/PJM_energy_datasets'
path = '.'
import sys
sys.path.insert(0,path)

full_set = DAYTON(path = ".",
                  seq_len = param_dset_lookback,
                  pred_horz = param_dset_forecast,
                  timestamp = False)
dytmax = full_set.max()
dytmin = full_set.min()
del(full_set)

train_set = DAYTON(path = ".",
                start_idx = 0, end_idx = 97036,
                seq_len = param_dset_lookback,
                pred_horz = param_dset_forecast,
                stride=29,
                timestamp = False)
val_set = DAYTON(path = ".",
              start_idx = 97036, end_idx = 97036+12129,
                seq_len = param_dset_lookback,
                pred_horz = param_dset_forecast,
                stride=param_dset_forecast,
                timestamp = False)
test_set = DAYTON(path = ".",
                start_idx = 97036+12129,
                seq_len = param_dset_lookback,
                pred_horz = param_dset_forecast,
                stride=param_dset_forecast,
                timestamp = False)


train_set.series = (train_set.series - dytmin)/(dytmax - dytmin)
val_set.series = (val_set.series - dytmin)/(dytmax - dytmin)
test_set.series = (test_set.series - dytmin)/(dytmax - dytmin)

print(train_set.series.shape)

torch.Size([3318, 840, 1])


In [9]:
#SWT Transformation
lev = 3

def swt_transformation(self_series):
    self_series = self_series.squeeze(-1)
    print("total series shape after squeeze: ", self_series.shape)
    self_series_numpy = self_series.numpy()
    print("torch shape: ", self_series.shape)
    print("numpy shape: ", self_series_numpy.shape)
#     print(self_series[0])
#     print(self_series_numpy[0])
    lev = 3
    da = data_preparation(self_series_numpy, self_series_numpy.shape[1], lev)
#     print(da[0][0])

    Vv = np.array(da)
    print(Vv.shape)
#     print(Vv[0][0])

    vv = Vv.reshape(Vv.shape[0],2*lev*Vv.shape[3])
    print(vv.shape)


    # dataset = scaler.fit_transform(vv)

    dat = vv.reshape(Vv.shape[0],2*lev,Vv.shape[3])
    print(dat.shape)

    return dat

train_set_transform = swt_transformation(train_set.series)
print("train_set_transform shape: ", train_set_transform.shape)
print("----------------------------------------------------")

val_set_transform = swt_transformation(val_set.series)
print("val_set_transform shape: ", val_set_transform.shape)
print("----------------------------------------------------")

test_set_transform = swt_transformation(test_set.series)
print("test_set_transform shape: ", test_set_transform.shape)
print("----------------------------------------------------")

total series shape after squeeze:  torch.Size([3318, 840])
torch shape:  torch.Size([3318, 840])
numpy shape:  (3318, 840)
Maximum decomposition level: 3


swt:   0%|          | 0/3318 [00:00<?, ?it/s]

(3318, 3, 2, 840)
(3318, 5040)
(3318, 6, 840)
train_set_transform shape:  (3318, 6, 840)
----------------------------------------------------
total series shape after squeeze:  torch.Size([471, 840])
torch shape:  torch.Size([471, 840])
numpy shape:  (471, 840)
Maximum decomposition level: 3


swt:   0%|          | 0/471 [00:00<?, ?it/s]

(471, 3, 2, 840)
(471, 5040)
(471, 6, 840)
val_set_transform shape:  (471, 6, 840)
----------------------------------------------------
total series shape after squeeze:  torch.Size([471, 840])
torch shape:  torch.Size([471, 840])
numpy shape:  (471, 840)
Maximum decomposition level: 3


swt:   0%|          | 0/471 [00:00<?, ?it/s]

(471, 3, 2, 840)
(471, 5040)
(471, 6, 840)
test_set_transform shape:  (471, 6, 840)
----------------------------------------------------


In [10]:
# Split seq_len and pred

trainX,trainY = train_set_transform[:int(train_set_transform.shape[0]*0.25),:,:param_dset_lookback], train_set_transform[:int(train_set_transform.shape[0]*0.25),:,param_dset_lookback:]
valX,valY = val_set_transform[:,:,:param_dset_lookback], val_set_transform[:,:,param_dset_lookback:]
testX,testY = test_set_transform[:,:,:param_dset_lookback], test_set_transform[:,:,param_dset_lookback:]

# trainY = trainY.reshape([trainY.shape[0], trainY.shape[1]*trainY.shape[2]])
# valY = valY.reshape([valY.shape[0], valY.shape[1]*valY.shape[2]])
# testY = testY.reshape([testY.shape[0], testY.shape[1]*testY.shape[2]])

print(trainX.shape)
print(valX.shape)
print(testX.shape)
print(trainY.shape)

trainX=np.transpose(trainX, (0, 2, 1))
valX =np.transpose(valX, (0, 2, 1))
testX=np.transpose(testX, (0, 2, 1))

trainY=np.transpose(trainY, (0, 2, 1))
valY =np.transpose(valY, (0, 2, 1))
testY=np.transpose(testY, (0, 2, 1))

print(trainX.shape)
print(valX.shape)
print(testX.shape)


(829, 6, 816)
(471, 6, 816)
(471, 6, 816)
(829, 6, 24)
(829, 816, 6)
(471, 816, 6)
(471, 816, 6)


## Model Initialization

In [11]:
# -*- coding: utf-8 -*-

import torch
import torch.nn as nn
import torch.nn.functional as F

# path1 = "/content/gdrive/MyDrive/FYP/Models"
import sys
sys.path.append("../../Models")

import math
import numpy as np
from itf_trainable import Itrainable
import copy
from Encodings.encoding import LearnablePositionalEncoding
from Encodings.encoding import CosineEncoding

from torch import Tensor
from typing import Optional

#TODO: breaks in eval mode with AttributeError: 'TBatchNorm' object has no attribute 'weight'
#Problem: in PyTorch transformer encoder layer base class, the sparsity fast path packs the weight and biase of
#         the norm layers into a tensor_args tuple.

class TBatchNorm(nn.Module):
    def __init__(self,num_features, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None):
        super(TBatchNorm,self).__init__()
        self.bn = nn.BatchNorm1d(num_features, eps, momentum, affine, track_running_stats, device, dtype)
        self.weight = self.bn.weight
        self.bias = self.bn.bias
        self.eps = self.bn.eps

    def forward(self,x):
        x = x.permute(0,2,1)
        x = self.bn(x)
        x = x.permute(0,2,1)
        return x

#Directly subclass from nn.BatchNorm1d so that references to weight and bias is inherited
# class TBatchNorm(nn.BatchNorm1d):
#     def __init__(self,num_features, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None):
#         super(TBatchNorm,self).__init__(num_features, eps, momentum, affine, track_running_stats, device, dtype)
#         # self.bn = nn.BatchNorm1d(num_features, eps, momentum, affine, track_running_stats, device, dtype)
#         # self.eps = self.bn.eps

#     def forward(self,x):
#         x = x.permute(0,2,1)
#         x = super().forward(x)
#         x = x.permute(0,2,1)
#         return x

In [12]:
class TransformerEncoderLayer_vis(nn.TransformerEncoderLayer):
    """torch.nn.TransformerEncoderLayer modified to retain the attention weights for
    later visualisation."""
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation=F.relu,
                 layer_norm_eps=1e-5, batch_first=False, norm_first=False,
                 device=None, dtype=None) -> None:
        super(TransformerEncoderLayer_vis,self).__init__(d_model, nhead, dim_feedforward,
                                                         dropout, activation,
                                                         layer_norm_eps, batch_first, norm_first,
                                                         device, dtype)
        self.attn_ = {}

    def _sa_block(self, x: Tensor,
                  attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x, attn_map = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=True)
        self.attn_['sa'] = attn_map.detach()
        return self.dropout1(x)


class TransformerDecoderLayer_vis(nn.TransformerDecoderLayer):
    """torch.nn.TransformerDecoderLayer modified to retain the attention weights for
    later visualisation."""
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation=F.relu,
                 layer_norm_eps=1e-5, batch_first=False, norm_first=False,
                 device=None, dtype=None) -> None:
        super(TransformerDecoderLayer_vis,self).__init__(d_model, nhead, dim_feedforward,
                                                         dropout, activation,
                                                         layer_norm_eps, batch_first, norm_first,
                                                         device, dtype)
        self.attn_ = {}

    def _sa_block(self, x: Tensor,
                  attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x, attn_map = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=True)
        self.attn_['sa'] = attn_map.detach()
        return self.dropout1(x)

    def _mha_block(self, x: Tensor, mem: Tensor,
                   attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x, attn_map = self.multihead_attn(x, mem, mem,
                                attn_mask=attn_mask,
                                key_padding_mask=key_padding_mask,
                                need_weights=True)
        self.attn_['mha'] = attn_map.detach()
        return self.dropout2(x)

In [13]:
class Transformer_Base(nn.Module):
    def __init__(self, seq_len=2560, out_seq_len=24, inp_dim=1, emb_dim=64,\
                 n_heads=4, n_enc_layers=2, n_dec_layers=2, ffdim=128,
                 pe_drop = 0.1,
                 ffdrop = 0.5,
                 attn_drop = 0.2):
        super(Transformer_Base,self).__init__()
        self.seq_len = seq_len
        self.out_seq_len = out_seq_len
        self.emb_dim = emb_dim

        self.input_linear = nn.Linear(inp_dim,emb_dim)

        self.pe = LearnablePositionalEncoding(emb_dim,dropout=pe_drop,max_len=seq_len)
        #self.pe = CosineEncoding(emb_dim,max_len=seq_len,scale_factor=0.5)

        drop_p = ffdrop
        self.trf_el = torch.nn.TransformerEncoderLayer(emb_dim,n_heads,ffdim,
                                                      activation=F.gelu,dropout=drop_p,
                                                      batch_first=True,norm_first=True)

        self.trf_dl = torch.nn.TransformerDecoderLayer(emb_dim,n_heads,ffdim,
                                activation=F.gelu,dropout=drop_p,
                                batch_first=True,norm_first=True)

        self.trf_el.norm1 = TBatchNorm(num_features=emb_dim) #nn.BatchNorm1d(num_features=seq_len)
        self.trf_el.norm2 = TBatchNorm(num_features=emb_dim) #nn.BatchNorm1d(num_features=seq_len)

        self.trf_dl.norm1 = TBatchNorm(num_features=emb_dim) #nn.BatchNorm1d(num_features=out_seq_len)
        self.trf_dl.norm2 = TBatchNorm(num_features=emb_dim)#nn.BatchNorm1d(num_features=out_seq_len)
        self.trf_dl.norm3 = TBatchNorm(num_features=emb_dim)#nn.BatchNorm1d(num_features=out_seq_len)

        # self.trf_e = nn.TransformerEncoder(self.trf_el,num_layers=n_enc_layers)
        # self.trf_d = nn.TransformerDecoder(self.trf_dl,num_layers=n_dec_layers)
        self.trf_e = None
        self.trf_d = None
        self.constructTransformer(n_enc_layers, n_dec_layers)

        self.out = nn.Linear(emb_dim,inp_dim)
        self.drop = nn.Dropout(p=drop_p)

    def constructTransformer(self,num_enclayers,num_declayers):
        self.trf_e = nn.TransformerEncoder(self.trf_el,num_layers=num_enclayers)
        self.trf_d = nn.TransformerDecoder(self.trf_dl,num_layers=num_declayers)

    def forward(self,x):
    #def forward(self,x, pred_gt = None):
        '''x: [batch dim, sequence length, variable dim]'''

        #Normalize input
        # E_x = x.mean(dim=-2,keepdim=True)
        # Var_x = x.var(dim=-2,keepdim=True)
        # sdx = torch.sqrt(Var_x+1e-6)
        # nx = (x-E_x)/sdx
        nx=x
        x1 = self.input_linear(nx) # x1.shape: [batch, seq len, emb dim]
        x1 =  self.pe(x1.permute(1,0,2)).permute(1,0,2)
        mem = self.trf_e(x1)

        iv = torch.zeros((x.shape[0],self.out_seq_len,x.shape[2]),device=mem.device)
        #pred_task_iv = self.dec_iv.broadcast_to((x.shape[0],self.out_seq_len,1))
        iv = self.input_linear(iv)
        iv = self.pe(iv.permute(1,0,2)).permute(1,0,2)
        #pred_task_iv = self.pe(pred_task_iv.permute(1,0,2)).permute(1,0,2)

        x2 = self.trf_d(iv,mem)
        o1 = self.out(x2)

        #Denormalize output
        # uo = sdx*o1 + E_x
        uo = o1
        return uo

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# path1 = "/content/gdrive/MyDrive/FYP/Models"
# path2 = "/content/gdrive/MyDrive/FYP/Models/BigBirdSparse"
# import sys
# sys.path.insert(1,path1)
# sys.path.insert(2,path2)
import sys
sys.path.append("../../Models")

from Encodings.encoding import LearnablePositionalEncoding
from BigBirdSparse.bb_sparse import BlockSparseMultiheadAttention
from BigBirdSparse.bb_fixed import BlockSparseFixedAttention
from transformer_base import TBatchNorm

class TransformerBBSparse(nn.Module):
    def __init__(self, seq_len=2560, out_seq_len=24, interval=1, inp_dim=1, emb_dim=64,\
                 n_heads=4, n_enc_layers=2, n_dec_layers=2, block_size=40,\
                ffdim=128, drop_p = 0.1):
        super(TransformerBBSparse,self).__init__()
        self.seq_len = seq_len
        self.out_seq_len = out_seq_len
        self.emb_dim = emb_dim
        #self.in_norm = nn.BatchNorm1d(seq_len,affine=False)
        self.input_linear = nn.Linear(inp_dim,emb_dim)
        #self.input_pool = nn.AvgPool1d(kernel_size=5,stride=5)
        self.pe = LearnablePositionalEncoding(emb_dim,dropout=0.1,max_len=seq_len)
        #self.pe = CosineEmbedding(emb_dim,max_len=seq_len,scale_factor=0.1)
        #self.tpe = TimestampCosineEmbedding(emb_dim, interval, max_len = seq_len)

        self.trf_el = nn.TransformerEncoderLayer(emb_dim,n_heads,ffdim,
                                                 activation=F.gelu,dropout=drop_p,
                                                 batch_first=True,norm_first=True)
        self.trf_el.self_attn = BlockSparseMultiheadAttention(emb_dim, n_heads, block_size,batch_first=True)
        #self.trf_el.self_attn = BlockSparseMheadAttnFixed(emb_dim, n_heads, block_size,batch_first=True)

        self.trf_dl = nn.TransformerDecoderLayer(emb_dim,n_heads,ffdim,
                                                 activation=F.gelu,dropout=drop_p,
                                                 batch_first=True,norm_first=True)

        self.trf_el.norm1 = TBatchNorm(num_features=emb_dim)
        self.trf_el.norm1 = TBatchNorm(num_features=emb_dim)
        self.trf_el.norm2 = TBatchNorm(num_features=emb_dim)

        self.trf_dl.norm1 = TBatchNorm(num_features=emb_dim)
        self.trf_dl.norm2 = TBatchNorm(num_features=emb_dim)
        self.trf_dl.norm3 = TBatchNorm(num_features=emb_dim)

        self.trf_e = nn.TransformerEncoder(self.trf_el,num_layers=n_enc_layers)
        self.trf_d = nn.TransformerDecoder(self.trf_dl,num_layers=n_dec_layers)

        # self.out = nn.Linear(emb_dim,inp_dim)
        self.out = nn.Linear(emb_dim,1)
        self.drop = nn.Dropout(p=drop_p)

        # self.dec_tar = torch.nn.Parameter(torch.zeros((out_seq_len,emb_dim)))
        # nn.init.normal_(self.dec_tar)
        self.aux_out = nn.Linear(emb_dim,1)#inp_dim)
        #self.reduce = nn.Linear(seq_len,out_seq_len)
        self.aux_in = nn.Linear(1,emb_dim)#inp_dim,emb_dim)

        self.expand = nn.Linear(seq_len,5120)
        self.compress = nn.Linear(5120,out_seq_len)

    def forward(self,x):
        '''x: [batch dim, sequence length, variable dim]'''
        x1 = self.input_linear(x) # x1.shape: [batch, seq len, emb dim]
        x1 = self.pe(x1.permute(1,0,2)).permute(1,0,2)
        #x1 = self.tpe(x1.permute(1,0,2),in_start_time).permute(1,0,2)
        mem = self.trf_e(x1)

        dec_in = self.aux_out(mem)
        dec_in = F.gelu(self.expand(dec_in.view((-1,self.seq_len))))
        dec_in = self.drop(dec_in)
        dec_in = self.compress(dec_in)
        #dec_in = F.gelu(self.reduce(dec_in.view((-1,self.seq_len))))

        dec_in = self.aux_in(dec_in.unsqueeze(-1))

        # x2 = self.trf_d(self.dec_tar.broadcast_to(x.shape[0],self.out_seq_len,self.emb_dim)\
        #                 ,mem)
        x2 = self.trf_d(self.pe(dec_in.permute(1,0,2))\
                        .permute(1,0,2),mem)
        # x2 = self.trf_d(self.tpe(dec_in.permute(1,0,2),pred_start_time)\
        #                 .permute(1,0,2),mem)

        o1 = self.out(x2)

        return o1



In [15]:
class TransformerBBFixed(nn.Module):
    def __init__(self, seq_len=2560, out_seq_len=24, interval=1, inp_dim=1, emb_dim=64,\
                 n_heads=4, n_enc_layers=2, n_dec_layers=2, block_size=40,\
                ffdim=128, drop_p = 0.1):
        super(TransformerBBFixed,self).__init__()
        self.seq_len = seq_len
        self.out_seq_len = out_seq_len
        self.emb_dim = emb_dim
        #self.in_norm = nn.BatchNorm1d(seq_len,affine=False)
        self.input_linear = nn.Linear(inp_dim,emb_dim)
        #self.input_pool = nn.AvgPool1d(kernel_size=5,stride=5)
        self.pe = LearnablePositionalEncoding(emb_dim,dropout=drop_p,max_len=seq_len)
        #self.pe = CosineEmbedding(emb_dim,max_len=seq_len,scale_factor=0.1)
        #self.tpe = TimestampCosineEmbedding(emb_dim, interval, max_len = seq_len)

        self.trf_el = nn.TransformerEncoderLayer(emb_dim,n_heads,ffdim,
                                                 activation=F.gelu,dropout=drop_p,
                                                 batch_first=True,norm_first=True)
        #self.trf_el.self_attn = BlockSparseMultiheadAttention(emb_dim, n_heads, block_size,batch_first=True)
        self.trf_el.self_attn = BlockSparseFixedAttention(emb_dim, n_heads, block_size,batch_first=True)

        self.trf_dl = nn.TransformerDecoderLayer(emb_dim,n_heads,ffdim,
                                                 activation=F.gelu,dropout=drop_p,
                                                 batch_first=True,norm_first=True)

        self.trf_el.norm1 = TBatchNorm(num_features=emb_dim)
        self.trf_el.norm1 = TBatchNorm(num_features=emb_dim)
        self.trf_el.norm2 = TBatchNorm(num_features=emb_dim)

        self.trf_dl.norm1 = TBatchNorm(num_features=emb_dim)
        self.trf_dl.norm2 = TBatchNorm(num_features=emb_dim)
        self.trf_dl.norm3 = TBatchNorm(num_features=emb_dim)

        self.trf_e = nn.TransformerEncoder(self.trf_el,num_layers=n_enc_layers)
        self.trf_d = nn.TransformerDecoder(self.trf_dl,num_layers=n_dec_layers)

        # self.out = nn.Linear(emb_dim,inp_dim)
        self.out = nn.Linear(emb_dim,1)
        self.drop = nn.Dropout(p=drop_p)

        # self.dec_tar = torch.nn.Parameter(torch.zeros((out_seq_len,emb_dim)))
        # nn.init.normal_(self.dec_tar)
        self.aux_out = nn.Linear(emb_dim,1)#inp_dim)
        #self.reduce = nn.Linear(seq_len,out_seq_len)
        self.aux_in = nn.Linear(1,emb_dim)#inp_dim,emb_dim)

        self.expand = nn.Linear(seq_len,5120)
        self.compress = nn.Linear(5120,out_seq_len)

    def forward(self,x):
        '''x: [batch dim, sequence length, variable dim]'''
        x1 = self.input_linear(x) # x1.shape: [batch, seq len, emb dim]
        x1 = self.pe(x1.permute(1,0,2)).permute(1,0,2)
        #x1 = self.tpe(x1.permute(1,0,2),in_start_time).permute(1,0,2)
        mem = self.trf_e(x1)

        dec_in = self.aux_out(mem)
        dec_in = F.gelu(self.expand(dec_in.view((-1,self.seq_len))))
        dec_in = self.drop(dec_in)
        dec_in = self.compress(dec_in)
        #dec_in = F.gelu(self.reduce(dec_in.view((-1,self.seq_len))))

        dec_in = self.aux_in(dec_in.unsqueeze(-1))

        # x2 = self.trf_d(self.dec_tar.broadcast_to(x.shape[0],self.out_seq_len,self.emb_dim)\
        #                 ,mem)
        x2 = self.trf_d(self.pe(dec_in.permute(1,0,2))\
                        .permute(1,0,2),mem)
        # x2 = self.trf_d(self.tpe(dec_in.permute(1,0,2),pred_start_time)\
        #                 .permute(1,0,2),mem)

        o1 = self.out(x2)

        return o1

In [16]:
# Early stopping
class EarlyStopper:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

class SaveBestModel:
    """
    Class to save or load the best model while training.
    """
    def __init__(
        self, best_valid_loss=float('inf')
    ):
        self.best_valid_loss = best_valid_loss

    def __call__(
        self, current_valid_loss, epoch, model
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
#             print(f"\nBest validation loss: {self.best_valid_loss}")
#             print(f"\nSaving best model for epoch: {epoch+1}\n")
            # Save the PyTorch model
            torch.save(model.state_dict(), 'transformer_5min.pth')

## Create model and data tensor

In [17]:
batch_size = 32

seq_len = param_dset_lookback
num_models = 2 * lev

# Transformer parameters
# inp_len = 2*lev
inp_len = 1
out_len = 2*lev
# emb_dim = 24
# n_heads = 4
emb_dim = 3 * inp_len # Must be 3 * inp_len
n_heads = 3 # divisible by emb_dim
enc_layer = 4
dec_layer = 4
ff_dim = 256
param_trf_bksz = 48

patience = 25
early_stopper = EarlyStopper(patience=patience, min_delta=0)
BestModels = []

for i in range(num_models):
    saveBestModel = SaveBestModel()
    BestModels.append(saveBestModel)


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

torch.manual_seed(42)

# Create multiple Pytorch models
Py_models = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



for i in range(num_models):
    # model = Transformer_Base(seq_len = param_dset_lookback,
    #                       # out_seq_len = param_dset_forecast,
    #                       out_seq_len = param_dset_forecast,
    #                       inp_dim = inp_len,
    #                       emb_dim = emb_dim,
    #                       n_heads = n_heads,
    #                       n_enc_layers = enc_layer,
    #                       n_dec_layers = dec_layer,
    #                       ffdim = ff_dim)
    model = TransformerBBFixed(seq_len = param_dset_lookback,
                          # out_seq_len = param_dset_forecast,
                          out_seq_len = param_dset_forecast,
                          inp_dim = inp_len,
                          emb_dim = emb_dim,
                          n_heads = n_heads,
                          n_enc_layers = enc_layer,
                          n_dec_layers = dec_layer,
                          block_size=param_trf_bksz,
                          ffdim = ff_dim)
    model.to(device)
    
    Py_models.append(model)

# Check classes are different
for i in range(num_models):
    print("Address of BestModel: ", id(BestModels[i]))
    print("Address of PyModel: ", id(Py_models[i]))


# Print model summary
# print(model)
def num_parameters(m):
    return sum([p.numel() for p in m.parameters()])

parameters = num_parameters(Py_models[0])

# print(f"Expected number of parameters: {m * dk * dk + m * 1 * 1 * n}")
print(f"Actual number of model parameters: {parameters}")

trainable_params = sum(
    p.numel() for p in Py_models[0].parameters() if p.requires_grad
)
print(f"Trainable model parameters: {trainable_params}" )

# Training loop
num_epochs = 100  # Replace with your desired number of epochs


for i in range(num_models):
    # Convert data to PyTorch tensors
    # Take the i value of SWT
    X_train, y_train = torch.tensor(trainX[:,:,i:i+1]), torch.tensor(trainY[:,:,i:i+1])
    X_val, y_val = torch.tensor(valX[:,:,i:i+1]), torch.tensor(valY[:,:,i:i+1])
    # print(X_train.shape)

    # Create DataLoader for training and validation data
    train_dataset = TensorDataset(X_train, y_train)
    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    
    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Optimizer and loss function
    optimizer = optim.RMSprop(model.parameters(), lr=0.001, eps=1e-07)
    # optimizer = optim.Adam(model.parameters())
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.MSELoss()

    # Train i value SWT model with early stoppage
    for epoch in tqdm(range(num_epochs), total= num_epochs, desc="epochs", position=0, leave=True):
        Py_models[i].train()
        train_loss = 0.0
        for inputs, targets in tqdm(train_loader, total=len(train_loader), desc="train batches", position=1, leave=True):
            inputs, targets = inputs.to(device).nan_to_num(), targets.to(device)
            optimizer.zero_grad()
            outputs = Py_models[i](inputs)
            print(outputs[0])
            print(targets[0])
    
            loss = nn.MSELoss(reduction='mean')(outputs[~targets.isnan()], targets[~targets.isnan()])
    #         print("targets: ", targets.shape)
    #         print("outputs: ", outputs.shape)
    #         loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
    
    #         for p in model.parameters():
    #             print(p.grad.norm())
    
            train_loss += loss.item()
    
        train_loss /= len(train_loader)
    
        Py_models[i].eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, total=len(val_loader), desc="val batches", position=2, leave=True):
                inputs, targets = inputs.to(device).nan_to_num(), targets.to(device)
                outputs = Py_models[i](inputs)
                val_loss += criterion(outputs, targets).item()
    
        val_loss /= len(val_loader)
        print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}')
    
        BestModels[i]
    
        # # Early stopping with patience
        # if early_stopper.early_stop(val_loss):
        #     print("Done!")
        #     break


Address of BestModel:  2067956435888
Address of PyModel:  2066001707312
Address of BestModel:  2067956436800
Address of PyModel:  2067956652912
Address of BestModel:  2067956442896
Address of PyModel:  2066089642448
Address of BestModel:  2067956443232
Address of PyModel:  2066089633648
Address of BestModel:  2067956447168
Address of PyModel:  2065921035808
Address of BestModel:  2067956447024
Address of PyModel:  2065921650752
Actual number of model parameters: 4327112
Trainable model parameters: 4327112


epochs:   0%|          | 0/100 [00:00<?, ?it/s]

train batches:   0%|          | 0/26 [00:00<?, ?it/s]

tensor([[-1.3021],
        [-1.6204],
        [-1.1917],
        [-1.0184],
        [-1.8573],
        [-1.2080],
        [-0.6737],
        [-1.4162],
        [-1.2923],
        [-0.5483],
        [-1.3852],
        [-1.5251],
        [-1.4203],
        [-1.0873],
        [-0.8743],
        [-1.1341],
        [-0.8230],
        [-1.4838],
        [-0.6863],
        [-1.1521],
        [ 0.2200],
        [-1.1321],
        [-1.4547],
        [-1.0255]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[1.0643],
        [0.9807],
        [0.8995],
        [0.8289],
        [0.7835],
        [0.7688],
        [0.7900],
        [0.8455],
        [0.9135],
        [0.9779],
        [1.0375],
        [1.0902],
        [1.1346],
        [1.1725],
        [1.2013],
        [1.2238],
        [1.2516],
        [1.2794],
        [1.3038],
        [1.3178],
        [1.3103],
        [1.2721],
        [1.2049],
        [1.1127]], device='cuda:0')
tensor([[-1.1409],
        [-0.3447],
        [-0

KeyboardInterrupt: 

In [ ]:
# Load the PyTorch model
loaded_model = Transformer_Base(seq_len = param_dset_lookback,
                          # out_seq_len = param_dset_forecast,
                          out_seq_len = param_dset_forecast,
                          inp_dim = inp_len,
                          emb_dim = emb_dim,
                          n_heads = n_heads,
                          n_enc_layers = enc_layer,
                          n_dec_layers = dec_layer,
                          ffdim = ff_dim)

loaded_model.load_state_dict(torch.load('transformer_5min.pth'))
loaded_model.to(device)
loaded_model.eval()

for i in range(num_models):
    # Convert data to PyTorch tensors
    X_train, y_train = torch.tensor(trainX[i]), torch.tensor(trainY[i])
    X_val, y_val = torch.tensor(valX[i]), torch.tensor(valY[i])
    X_test, y_test = torch.tensor(testX), torch.tensor(testY)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


# Use the whole signal (both train and validation data)
# The metrics are computed only using the validation part.
# This is needed for the signal processing
print(testX.shape)

# Testing the model on the test dataset
test_outputs = []
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in tqdm(test_loader, total=len(test_loader), desc="test batch"):
        inputs, targets = inputs.to(device).nan_to_num(), targets.to(device)
        outputs = loaded_model(inputs)
        test_outputs.append(outputs.cpu().numpy())  # Collect the outputs

        loss = nn.MSELoss(reduction='mean')(outputs[~targets.isnan()], targets[~targets.isnan()])
        test_loss += loss
#         test_loss += criterion(outputs, targets).item()

testPredict_a = np.concatenate(test_outputs, axis=0)  # Concatenate outputs into a single numpy array
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

# Result from test
testPredict_a = np.transpose(testPredict_a, (0, 2, 1))
print(testPredict_a.shape)

testX_original = np.transpose(testX, (0, 2, 1))
print(testX_original.shape)

test_cat = np.concatenate((testX_original, testPredict_a), axis=2)
print(test_cat.shape)

In [ ]:

D = test_cat.reshape(test_cat.shape[0],test_cat.shape[1]*test_cat.shape[2])
print(D.shape)

R = D.reshape(test_cat.shape[0],lev,2,test_cat.shape[2])
print(R.shape)

R = data_organization(R)

# print(R)

re=data_reconstruction(R)
Re = np.array(re)
print(Re.shape)


# Re = np.nan_to_num(Re)
Re = torch.from_numpy(Re)

In [ ]:
test_set_squeeze = test_set.series.squeeze(-1)
print("test truth: ", test_set_squeeze.shape)

test_rmse = torch.nn.MSELoss(reduction='none')(test_set_squeeze[:,param_dset_lookback:], Re[:,param_dset_lookback:]).nanmean(dim=-2).sqrt_()

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt


# test_set_squeeze = test_set.series.squeeze(-1)
# print("test truth: ", test_set_squeeze.shape)

# print(test_set_squeeze[:,param_dset_lookback:].shape)
# print(Re[:,param_dset_lookback:].shape)


# test_rmse = math.sqrt( mean_squared_error(test_set_squeeze[:,param_dset_lookback:], Re[:,param_dset_lookback:]))
# # test_rmse = math.sqrt( mean_squared_error(test_compare, Re))

# test_mae=mean_absolute_error(test_set_squeeze[:,param_dset_lookback:], Re[:,param_dset_lookback:])
# # test_mae=mean_absolute_error(test_compare, Re)

# # mape=100*np.mean(np.divide(abs(test_compare[:,1632:]- Re[:,1632:]),test_compare[:,1632:]))

test_actual = test_set_squeeze[:,param_dset_lookback:]
test_predict = Re[:,param_dset_lookback:]

test_rmse = torch.nn.MSELoss(reduction='none')(test_actual, test_predict).nanmean().sqrt_()

test_mae = (test_predict-test_actual).abs_().nanmean()

mape = (2*(test_actual-test_predict).abs_() / (test_actual.abs() + test_predict.abs())).nanmean()

In [ ]:
print('RMSE:  %.6f' % test_rmse)
print('MAE:  %.6f' % test_mae)
print('MAPE:  %.6f' % mape)

In [ ]:
fig = plt.figure()
# print(test_set_squeeze[:3,param_dset_lookback:])
# print(Re[:3,param_dset_lookback:])
plt.plot(test_set_squeeze[:1,param_dset_lookback:][0])
plt.plot(Re[:1,param_dset_lookback:][0])
plt.xlabel('Time/min')
plt.ylabel('Electricity load (kWh)')
plt.legend(['True', 'Predict'], loc='upper left')
plt.show()
